# The sign function

The [sign function](https://en.wikipedia.org/wiki/Sign_function) of a real number $x$ is:

\begin{align}
    sgn(x)=
    \begin{cases}
        -1, &\text{if } x < 0\\
        0, &\text{if } x = 0\\
        1, &\text{if } x > 0
    \end{cases}
\end{align}

<!-- The sign function is related to the Heaviside step function, $H(x)$, as follows: $H(x) = \frac{1}{2}+\frac{1}{2}sgn(x)$. $H(x)$ can be used as indicator function, and two indicator functions are enough to determine the sign of a number, i.e., $\mathbf{1}_{x>0}$ and $\mathbf{1}_{x<0}$. As such, -->

Any protocol that can evaluate greater than (GT) can be extended to evalute the sign function. There are a number of GT protocols and some efficient ones will be explored next.

Note that there are papers describing sgn() evaluation using HE only (e.g., [ref](https://arxiv.org/pdf/1810.12380.pdf)).

## The Lin Tzeng protocol

The Lin Tzeng protocol is detailed [here](https://doi.org/10.1007%2F11496137_31). It reduces the GT problem to a set intersection problem, which is easy to compute in a privacy preserving way.

* Let $s=s_n s_{n-1} \ldots s_1 \in \{0,1\}^n$ be a binary string of length $n$.
* Denote 0-encoding of $s$ as the set $S^0_s=\{s_n s_{n-1} \ldots s_{i+1} 1 \mid s_i = 0, 1 \leq i \leq n\}$
* Denote 1-encoding of $s$ as the set $S^1_s=\{s_n s_{n-1} \ldots s_{i} \mid s_i = 1, 1 \leq i \leq n\}$

Given two values $(x,y)$, $x > y$ if and only if $S^1_x \cap S^0_y \neq \emptyset$, otherwise $x \leq y$.

### Example of the Lin Tzeng protocol with $(x,y)\in\mathbb{N}_0$

In [1]:
import numpy as np

In [2]:
def num_to_arr(n, length=None, print_flag=False):
    '''
    Takes as input a non negative integer
    Returns bin() representation in an array padded with leading zeros up to provided length
    '''
    arr = list(bin(n)[2:].zfill(length)) if length else list(bin(n)[2:])
    if print_flag: print('Binary stream of {}: '.format(n), bin(n)[2:].zfill(length)) if length else list(bin(n)[2:])
    return arr

def encoding(n, type, length=None, print_flag=False):
    '''
    Takes as input binary array
    Returns 0 or 1 encoding of array
    '''
    assert (type==0) or (type==1), "Type must be 0 or 1"
    assert n >= 0, 'Number must be non negative integer'
    arr = num_to_arr(n, length, print_flag)
    indices = np.where(np.array(arr) == str(type))[0]
    s = set()
    for i in indices:
        encoding = ''.join(arr[:i]+['1'])
        s.add(encoding)
    if print_flag: print('{}-encoding of {}: {}'.format(type, n, s))
    return(s)

def max_length(a, b):
    return max(len(num_to_arr(a)), len(num_to_arr(b)))

def yao_compare(a, b, length=None, print_flag=False):
    '''
    Takes two non zero integers as input
    Returns 
        * 1-encoding of a
        * 0-encoding of b
        * True if a <= b
    '''
    l = length if length else max_length(a, b)
    s_a_1 = encoding(a, type=1, length=l, print_flag=print_flag)
    s_b_0 = encoding(b, type=0, length=l, print_flag=print_flag)
    return s_a_1, s_b_0, s_a_1.intersection(s_b_0)==set()

In [41]:
# a, b = 50, 13
# _, _, less_or_equal = yao_compare(a, b, print_flag=True)
# print('Result of the comparison: ', 'a <= b' if less_or_equal else 'a > b')

In [46]:
from ecpy.curves import Curve
from Crypto.Hash import SHA256
from Crypto.Util import number
import random

In [31]:
def sha256(m):
    '''
    Takes input m
    Ouputs SHA256(m)
    '''
    m = m.encode('utf-8')
    H = SHA256.new()
    H.update(m) 
    return H.hexdigest()

In [8]:
curve = Curve.get_curve('Curve25519')
G = curve.generator
order = curve.order

In [60]:
# Alice gets her encoding and computes the encrypted set for each
a = 64865654654
b = 54000000000
s_1_a, s_0_b, less_or_equal = yao_compare(a, b, print_flag=False)

# We need a way to create unique mappings from the encoding sets to a point multiplication set
# We cannot use int() because int('00111') == int('0111'). 
# We can either increment each char by 1 or we simply hash the values
# I pick hashing because it is more convential and it is easier to implement

alice_values = [int(sha256(x), base=16) for x in s_1_a]
bob_values = [int(sha256(x), base=16) for x in s_0_b]

r = number.getRandomNBitInteger(256) # Alice random
s = number.getRandomNBitInteger(256) # Bob random

raAi = [r * alice_values[i] * G for i in range(len(alice_values))]
sraAi = [s * raAi[i] for i in range(len(raAi))]

inv_r = pow(r, -1, order)
random.shuffle(sraAi)

sAi = [inv_r * sraAi[i] for i in range(len(sraAi))]  
sBi = [s * bob_values[i] * G for i in range(len(bob_values))]
len(np.intersect1d([sBi[i].x for i in range(len(sBi))], [sAi[i].x for i in range(len(sAi))]))

1

### Notes

The two parties can agree on a max bit length beforehand. In the HEPIP case the max bit length is given by the accuracy of the GPS and the min max values [-90, 90] for lattitude and [-180, 180] for longitude. 
Once the encoding is done, we can now rely on a PSI protocol for the final comparison. See https://github.com/mcoder/private-set-intersection for possible options. Also see [this paper](https://eprint.iacr.org/2019/723.pdf) that is the basis for Google's PSI protocol called "join and compute".